In [ ]:
# Improve by ChatGPT & GitHub Copilot
# Running the requierements.ipynb
%run /bettik/PROJECTS/pr-data-ocean/riverama/Notebooks/requierements.ipynb

# Running functions_IT.ipynb
%run /bettik/PROJECTS/pr-data-ocean/riverama/Notebooks/OSSE_borrador/IT_modes/functions_IT_modes_v1.ipynb

In [ ]:
ds = xr.open_mfdataset("/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_it/analysis/ssh_it_cart_*.nc",combine='nested',concat_dim='time_counter',parallel=True)

In [ ]:
ds

In [ ]:
ssh_it = ds.sel(time_counter='2014-05-01T00:30:00') # Select one time step

In [ ]:
k1 = 0.00252
k2 = 0.00964
k3 = 0.02100
k4 = 0.03100

nx = ssh_it['x'].size
ny = ssh_it['y'].size
dx = 2  #km

kx = np.fft.fftfreq(3*nx,dx) # km
ky = np.fft.fftfreq(3*ny,dx) # km
k, l = np.meshgrid(kx,ky)
wavenum2D = np.sqrt(k**2 + l**2)

wavenumbers_mode_1 = [0.5*k1,0.5*(k1+k2)] # in km
wavenumbers_mode_2 = [0.5*(k1+k2),0.5*(k2+k3)] # in km
wavenumbers_mode_3 = [0.5*(k2+k3),0.5*(k3+k4)] # in km

In [ ]:
# Apply the bandpass filters
bandpass_mode_1 = bandpass(wavenumbers=wavenumbers_mode_1, nx=nx, ny=ny, wavenum2D=wavenum2D)
bandpass_mode_2 = bandpass(wavenumbers=wavenumbers_mode_2, nx=nx, ny=ny, wavenum2D=wavenum2D)
bandpass_mode_3 = bandpass(wavenumbers=wavenumbers_mode_3, nx=nx, ny=ny, wavenum2D=wavenum2D)

In [ ]:
# Create the spatial window
window = create_spatial_window(nx=nx, ny=ny)

# Parallel processing to extract modes for single time step data
results = Parallel(n_jobs=-1, backend='multiprocessing')(delayed(extract_it_mode)(
    ssh_it['ssh_it_cart'].values, window, dx, bandpass_mode_1, bandpass_mode_2, bandpass_mode_3) for _ in range(1)
)

# Extract individual modes directly from the results
ssh_filtered_1, ssh_filtered_2, ssh_filtered_3 = results[0]

In [ ]:
# Create a new xarray.Dataset
ssh_it_modes = xr.Dataset(
    {
        "ssh_it_mode_1": (("y", "x"), ssh_filtered_1),
        "ssh_it_mode_2": (("y", "x"), ssh_filtered_2),
        "ssh_it_mode_3": (("y", "x"), ssh_filtered_3),
    },
    coords={
        "y": ("y", ssh_it["y"].values),
        "x": ("x", ssh_it["x"].values),
        "time_counter": ssh_it["time_counter"].values
    }
)

In [ ]:
# Get the date for the title
date = str(ssh_it_modes.time_counter.values)
formatted_date = pd.to_datetime(str(date)).strftime('%Y-%m-%d %H:%M')

# Plotting
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

# Titles for each subplot
mode_titles = ['Mode 1', 'Mode 2', 'Mode 3']

for i, mode in enumerate(ssh_it_modes.data_vars):
    im = axs[i].pcolormesh(ssh_it_modes[mode], cmap=cmocean.cm.balance, vmin=-0.1, vmax=0.1)
    axs[i].set_title(mode_titles[i])
    fig.colorbar(im, ax=axs[i], orientation='vertical', shrink=0.84)
    axs[i].set_aspect('equal')  # Set the aspect ratio to be equal

# Set the main title
fig.suptitle(f'Internal Tides at {formatted_date}', fontsize=16)

plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to make room for the main title
plt.show()

In [ ]:
# Load the dataset
ds = xr.open_mfdataset("/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_it/analysis/ssh_it_cart_*.nc",
                       combine='nested', concat_dim='time_counter', parallel=True).load()

# Define parameters
k1 = 0.00252
k2 = 0.00964
k3 = 0.02100
k4 = 0.03100

dx = 2  # km

# Define wavenumbers for modes
wavenumbers_mode_1 = [0.5 * k1, 0.5 * (k1 + k2)]  # in km
wavenumbers_mode_2 = [0.5 * (k1 + k2), 0.5 * (k2 + k3)]  # in km
wavenumbers_mode_3 = [0.5 * (k2 + k3), 0.5 * (k3 + k4)]  # in km

# Directory to save frames
output_dir = "/bettik/PROJECTS/pr-data-ocean/riverama/Frames/IT_Modes"
os.makedirs(output_dir, exist_ok=True)

# Time range for processing Internal Tides at 2014-05-09 03:30 3270
time_range = pd.date_range(start='2014-05-09T04:30:00', end='2014-05-31T00:30:00', freq='0.5H')
frame_counter = 3271

for current_date in time_range:
    # Select the time step using the nearest available time
    ssh_it = ds.sel(time_counter=current_date, method='nearest')
    
    nx = ssh_it['x'].size
    ny = ssh_it['y'].size
    
    kx = np.fft.fftfreq(3 * nx, dx)  # km
    ky = np.fft.fftfreq(3 * ny, dx)  # km
    k, l = np.meshgrid(kx, ky)
    wavenum2D = np.sqrt(k ** 2 + l ** 2)
    
    # Apply the bandpass filters
    bandpass_mode_1 = bandpass(wavenumbers=wavenumbers_mode_1, nx=nx, ny=ny, wavenum2D=wavenum2D)
    bandpass_mode_2 = bandpass(wavenumbers=wavenumbers_mode_2, nx=nx, ny=ny, wavenum2D=wavenum2D)
    bandpass_mode_3 = bandpass(wavenumbers=wavenumbers_mode_3, nx=nx, ny=ny, wavenum2D=wavenum2D)
    
    # Create the spatial window
    window = create_spatial_window(nx=nx, ny=ny)
    
    # Parallel processing to extract modes for single time step data
    results = Parallel(n_jobs=-1, backend='multiprocessing')(delayed(extract_it_mode)(
        ssh_it['ssh_it_cart'].values, window, dx, bandpass_mode_1, bandpass_mode_2, bandpass_mode_3) for _ in range(1)
    )
    
    # Extract individual modes directly from the results
    ssh_filtered_1, ssh_filtered_2, ssh_filtered_3 = results[0]
    
    # Create a new xarray.Dataset
    ssh_it_modes = xr.Dataset(
        {
            "ssh_it_mode_1": (("y", "x"), ssh_filtered_1),
            "ssh_it_mode_2": (("y", "x"), ssh_filtered_2),
            "ssh_it_mode_3": (("y", "x"), ssh_filtered_3),
        },
        coords={
            "y": ("y", ssh_it["y"].values),
            "x": ("x", ssh_it["x"].values),
            "time_counter": ssh_it["time_counter"].values
        }
    )
    
    # Get the date for the title
    date = str(ssh_it_modes.time_counter.values)
    formatted_date = pd.to_datetime(str(date)).strftime('%Y-%m-%d %H:%M')
    
    # Plotting
    fig, axs = plt.subplots(1, 3, figsize=(18, 6))
    
    # Titles for each subplot
    mode_titles = ['Mode 1', 'Mode 2', 'Mode 3']
    
    for i, mode in enumerate(ssh_it_modes.data_vars):
        im = axs[i].pcolormesh(ssh_it_modes[mode], cmap=cmocean.cm.balance, vmin=-0.1, vmax=0.1)
        axs[i].set_title(mode_titles[i])
        fig.colorbar(im, ax=axs[i], orientation='vertical', shrink=0.84)
        axs[i].set_aspect('equal')  # Set the aspect ratio to be equal
    
    # Set the main title
    fig.suptitle(f'Internal Tides at {formatted_date}', fontsize=16)
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to make room for the main title
    
    # Save the plot
    output_file = os.path.join(output_dir, f'frame_{frame_counter}.png')
    plt.savefig(output_file)
    plt.close(fig)
    
    # Increment the frame counter
    frame_counter += 1

print("ta da!")